In [1]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

abalone_train.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


In [3]:
abalone_features = abalone_train.copy()
abalone_labels = abalone_features.pop('Age')

In [4]:
abalone_features = np.array(abalone_features)
abalone_features

array([[0.435, 0.335, 0.11 , ..., 0.136, 0.077, 0.097],
       [0.585, 0.45 , 0.125, ..., 0.354, 0.207, 0.225],
       [0.655, 0.51 , 0.16 , ..., 0.396, 0.282, 0.37 ],
       ...,
       [0.53 , 0.42 , 0.13 , ..., 0.374, 0.167, 0.249],
       [0.395, 0.315, 0.105, ..., 0.118, 0.091, 0.119],
       [0.45 , 0.355, 0.12 , ..., 0.115, 0.067, 0.16 ]])

In [5]:
abalone_model = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
])

abalone_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

Metal device set to: Apple M1


2022-07-14 09:48:32.571924: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-14 09:48:32.572076: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
abalone_model.fit(abalone_features, abalone_labels, epochs=10)

Epoch 1/10


2022-07-14 09:48:32.626478: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 29/104 [=======>......................] - ETA: 0s - loss: 89.6008

2022-07-14 09:48:32.832224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


104/104 [==============================] - 1s 4ms/step - loss: 61.0565
Epoch 2/10
104/104 [==============================] - 0s 3ms/step - loss: 12.1980
Epoch 3/10
104/104 [==============================] - 0s 4ms/step - loss: 8.9082
Epoch 4/10
104/104 [==============================] - 0s 3ms/step - loss: 8.3427
Epoch 5/10
104/104 [==============================] - 0s 4ms/step - loss: 7.8387
Epoch 6/10
104/104 [==============================] - 0s 4ms/step - loss: 7.4212
Epoch 7/10
104/104 [==============================] - 0s 4ms/step - loss: 7.0960
Epoch 8/10
104/104 [==============================] - 0s 4ms/step - loss: 6.8460
Epoch 9/10
104/104 [==============================] - 0s 4ms/step - loss: 6.7018
Epoch 10/10
104/104 [==============================] - 0s 3ms/step - loss: 6.5780


In [7]:
normalize = layers.Normalization()

In [8]:
normalize.adapt(abalone_features)

2022-07-14 09:48:36.721540: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-14 09:48:36.736026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
norm_abalone_model = tf.keras.Sequential([
    normalize,
    layers.Dense(64),
    layers.Dense(1)
])

norm_abalone_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                           optimizer = tf.keras.optimizers.Adam())

norm_abalone_model.fit(abalone_features, abalone_labels, epochs=10)

Epoch 1/10
 29/104 [=======>......................] - ETA: 0s - loss: 100.5450

2022-07-14 09:48:37.062865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


104/104 [==============================] - 1s 4ms/step - loss: 92.4463
Epoch 2/10
104/104 [==============================] - 0s 4ms/step - loss: 53.2760
Epoch 3/10
104/104 [==============================] - 0s 3ms/step - loss: 16.2204
Epoch 4/10
104/104 [==============================] - 0s 4ms/step - loss: 5.9526
Epoch 5/10
104/104 [==============================] - 0s 4ms/step - loss: 5.2108
Epoch 6/10
104/104 [==============================] - 0s 4ms/step - loss: 5.1507
Epoch 7/10
104/104 [==============================] - 0s 4ms/step - loss: 5.1174
Epoch 8/10
104/104 [==============================] - 0s 4ms/step - loss: 5.0709
Epoch 9/10
104/104 [==============================] - 0s 4ms/step - loss: 5.0414
Epoch 10/10
104/104 [==============================] - 0s 4ms/step - loss: 5.0119


###### Mixed Data Type

In [10]:
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [11]:
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')

In [12]:
# # Create symbolic Input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Perform a calculation using the input
result = 2*input + 1

# the result doesn't have a value
result

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.__operators__.add')>

In [13]:
calc = tf.keras.Model(inputs=input, outputs=result)

In [14]:
print(calc(1))
print(calc(2))

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


make pandas datatype to dictionary of input layer of tensorflow datatype

In [15]:
inputs = {}

for name, column in titanic_features.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32
    
    inputs[name] = tf.keras.Input(shape=(1, ), name=name, dtype=dtype)

inputs

{'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'age')>,
 'n_siblings_spouses': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'n_siblings_spouses')>,
 'parch': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'parch')>,
 'fare': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fare')>,
 'class': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'class')>,
 'deck': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'deck')>,
 'embark_town': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'embark_town')>,
 'alone': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'alone')>}

check out numeric data and normalize

In [16]:
numeric_inputs = {name:input for name, input in inputs.items() if input.dtype == tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
# # later, check that change titanic_feature[name] to numeric_inputs[name]
norm.adapt(np.array(titanic_features[numeric_inputs.keys()]))

all_numeric_inputs = norm(x)
all_numeric_inputs

2022-07-14 09:48:41.297341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-14 09:48:41.311280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization_1')>